In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque
from sklearn.neighbors import NearestNeighbors

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
df1 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2018-2019.csv')
df1.Date = pd.to_datetime(df1.Date)

df2 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2019-2020.csv')
df2.Date = pd.to_datetime(df2.Date)

dfs1 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2018-2019.csv')
dfs1.Date = pd.to_datetime(dfs1.Date)

dfs2 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2019-2020.csv')
dfs2.Date = pd.to_datetime(dfs2.Date)

df = df1.append(df2)
df.reset_index(drop=True, inplace=True)

df_small = dfs1.append(dfs2)
df_small.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2018-07-30 07:00:00   1.31106   1.31109  1.30964    1.31094   1.31131   
1 2018-07-30 07:15:00   1.31094   1.31184  1.31091    1.31160   1.31112   
2 2018-07-30 07:30:00   1.31162   1.31291  1.31161    1.31288   1.31181   
3 2018-07-30 07:45:00   1.31294   1.31380  1.31277    1.31289   1.31312   
4 2018-07-30 08:00:00   1.31290   1.31324  1.31226    1.31229   1.31304   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.31138  1.30981    1.31111   1.31118   1.31124  1.30972    1.31102  
1   1.31202  1.31107    1.31180   1.31103   1.31192  1.31099    1.31170  
2   1.31310  1.31179    1.31306   1.31172   1.31300  1.31170    1.31297  
3   1.31415  1.31293    1.31306   1.31303   1.31396  1.31285    1.31298  
4   1.31342  1.31243    1.31245   1.31297   1.31333  1.31234    1.31237  
---------------------------------------------------------------------------
                     Date  Bi

In [5]:
print(df_small.head())
print('-' * 75)
print(df_small.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2018-07-30 07:00:00   1.31106   1.31109  1.31061    1.31069   1.31131   
1 2018-07-30 07:05:00   1.31075   1.31089  1.30964    1.31032   1.31093   
2 2018-07-30 07:10:00   1.31038   1.31102  1.31038    1.31094   1.31054   
3 2018-07-30 07:15:00   1.31094   1.31136  1.31091    1.31098   1.31112   
4 2018-07-30 07:20:00   1.31103   1.31150  1.31103    1.31134   1.31121   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.31138  1.31077    1.31086   1.31118   1.31124  1.31070    1.31078  
1   1.31107  1.30981    1.31047   1.31084   1.31098  1.30972    1.31040  
2   1.31119  1.31054    1.31111   1.31046   1.31110  1.31046    1.31102  
3   1.31155  1.31107    1.31114   1.31103   1.31145  1.31099    1.31106  
4   1.31169  1.31121    1.31152   1.31112   1.31160  1.31112    1.31143  
---------------------------------------------------------------------------
                      Date  B

In [6]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=14)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [7]:
df_small['ema200'] = talib.EMA(df_small['Mid_Close'], timeperiod=200)
df_small['adx'] = talib.ADX(df_small['Mid_High'], df_small['Mid_Low'], df_small['Mid_Close'], timeperiod=14)
df_small['rsi'] = talib.RSI(df_small['Mid_Close'], timeperiod=14)
df_small.dropna(inplace=True)
df_small.reset_index(drop=True, inplace=True)

In [8]:
value_per_pip = 1.0
max_pips_to_risk = 0.0100
amounts_per_day = [-4, -5, -6]

In [9]:
class AatTuple:
    def __init__(self, time, pips_risked, pips_gained, total_profit, adx_assumption, ema_assumption, rsi_assumption):
        self.time = time
        self.pips_risked = pips_risked
        self.pips_gained = pips_gained
        self.total_profit = total_profit
        self.adx_assumption = adx_assumption
        self.ema_assumption = ema_assumption
        self.rsi_assumption = rsi_assumption

In [10]:
training_data = []

In [11]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_training_simulation(risk_reward_ratio, pullback_cushion, bar_length, adx_cutoff, spread_cutoff, max_open_trades=1, lam=0.5):
    pullback_cushion /= 10000
    spread_cutoff /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trades = deque(maxlen=max_open_trades)

    def iterate_thru_small_data(trade):
        def get_end_date(j):
          return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        j = df_small[df_small.Date >= trade['start_date']].index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak

        current_training_points = []
        step = 0

        while j < len(df_small):
          curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_date, curr_adx, curr_ema, curr_rsi, curr_ml, curr_mh = \
          df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                         'Ask_Close', 'Date', 'adx', 'ema200', 'rsi', 'Mid_Low', 'Mid_High']]

          if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            for point in current_training_points:
              point.total_profit = reward + day_fees

            break

          # if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
          #     trade['stop_loss'] = curr_bid_high - trade['pips_risked']

          if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            for point in current_training_points:
              point.total_profit = reward + day_fees

            break

          if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            for point in current_training_points:
              point.total_profit = reward + day_fees

            break

          # if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
          #   trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

          if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            for point in current_training_points:
              point.total_profit = reward + day_fees

            break

          pips_gained_so_far = (curr_bid_close - trade['open_price']) * trade['n_units'] if trade['trade_type'] == 'buy' else (trade['open_price'] - curr_ask_close) * trade['n_units']

          z_adx_prev = 0 if len(current_training_points) == 0 else current_training_points[-1].adx_assumption * lam
          z_ema_prev = 0 if len(current_training_points) == 0 else current_training_points[-1].ema_assumption * lam
          z_rsi_prev = 0 if len(current_training_points) == 0 else current_training_points[-1].rsi_assumption * lam

          adx_holds = int(curr_adx > adx_cutoff)
          ema_holds = int(curr_ml > curr_ema if trade['trade_type'] == 'buy' else curr_mh < curr_ema)
          rsi_holds = int(curr_rsi < 70 if trade['trade_type'] == 'buy' else curr_rsi > 30)

          adx_assumption = z_adx_prev + ((1 - lam) * adx_holds)
          ema_assumption = z_ema_prev + ((1 - lam) * ema_holds)
          rsi_assumption = z_rsi_prev + ((1 - lam) * rsi_holds)

          current_training_points.append(AatTuple(step, trade['pips_risked'], pips_gained_so_far, 0, adx_assumption, ema_assumption, rsi_assumption))

          step += 1
          j += 1
        
        training_data.extend(current_training_points)

    i = 2

    while i < len(df):          
        curr_date = df.loc[df.index[i], 'Date']

        # Remove any expired trades -----------------------------------------
        trades_to_remove = []

        for trade in trades:
          if trade['end_date'] is not None and curr_date >= trade['end_date']:
            trades_to_remove.append(trade)

        for trade in trades_to_remove:
          trades.remove(trade)
        # -------------------------------------------------------------------

        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        # enough_volatility = (spread / atr) <= atr_percentage
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        trending = adx >= adx_cutoff

        process_trade = False

        if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml > ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_buys = True

            for trade in trades:
                if trade['trade_type'] != 'buy':
                    all_buys = False
                    break

            if all_buys:
              pullback = curr_ao - pullback_cushion

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_ask_open)
                  stop_loss = round(pullback, 5)
                  # stop_loss = round(pullback - pullback_cushion, 3)

                  if stop_loss < open_price:
                      curr_pips_to_risk = open_price - stop_loss

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                            'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_buys += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh < ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_sells = True

            for trade in trades:
                if trade['trade_type'] != 'sell':
                    all_sells = False
                    break

            if all_sells:
              pullback = curr_bo + pullback_cushion

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_bid_open)
                  stop_loss = round(pullback, 5)
                  # stop_loss = round(pullback + pullback_cushion, 3)

                  if stop_loss > open_price:
                      curr_pips_to_risk = stop_loss - open_price

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                  'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                  'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_sells += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        if process_trade:
          iterate_thru_small_data(trades[-1])

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak

In [12]:
# ----------------------------------------------------------------------------------------------------
# Run training simulation
# ----------------------------------------------------------------------------------------------------
reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak = run_training_simulation(2, 20, 5, 30, 3)

print(reward)
print('Num buys: ' + str(n_sells))
print('Num sells: ' + str(n_buys))
print('Num trades: ' + str(n_buys + n_sells))
print('Num wins: ' + str(n_wins))
print('Num losses: ' + str(n_losses))
print('Win streak: ' + str(win_streak))
print('Loss streak: ' + str(loss_streak))

-1087.0000000000014
Num buys: 22
Num sells: 15
Num trades: 37
Num wins: 9
Num losses: 28
Win streak: 2
Loss streak: 7


In [13]:
len(training_data)

4054

In [14]:
# Repeate, but with test data
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_small = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small.reset_index(drop=True, inplace=True)

In [15]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=14)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [16]:
df_small['ema200'] = talib.EMA(df_small['Mid_Close'], timeperiod=200)
df_small['adx'] = talib.ADX(df_small['Mid_High'], df_small['Mid_Low'], df_small['Mid_Close'], timeperiod=14)
df_small['rsi'] = talib.RSI(df_small['Mid_Close'], timeperiod=14)
df_small.dropna(inplace=True)
df_small.reset_index(drop=True, inplace=True)

In [17]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_regular_simulation(risk_reward_ratio, pullback_cushion, bar_length, adx_cutoff, spread_cutoff, max_open_trades=1):
    pullback_cushion /= 10000
    spread_cutoff /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trades = deque(maxlen=max_open_trades)

    def iterate_thru_small_data(trade):
        def get_end_date(j):
          return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        j = df_small[df_small.Date >= trade['start_date']].index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak

        while j < len(df_small):
          curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_date, curr_adx, curr_ema, curr_rsi, curr_ml, curr_mh = \
          df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                         'Ask_Close', 'Date', 'adx', 'ema200', 'rsi', 'Mid_Low', 'Mid_High']]

          if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          # if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
          #     trade['stop_loss'] = curr_bid_high - trade['pips_risked']

          if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          # if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
          #   trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

          if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          j += 1

    i = 2

    while i < len(df):          
        curr_date = df.loc[df.index[i], 'Date']

        # Remove any expired trades -----------------------------------------
        trades_to_remove = []

        for trade in trades:
          if trade['end_date'] is not None and curr_date >= trade['end_date']:
            trades_to_remove.append(trade)

        for trade in trades_to_remove:
          trades.remove(trade)
        # -------------------------------------------------------------------

        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        # enough_volatility = (spread / atr) <= atr_percentage
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        trending = adx >= adx_cutoff

        process_trade = False

        if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml > ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_buys = True

            for trade in trades:
                if trade['trade_type'] != 'buy':
                    all_buys = False
                    break

            if all_buys:
              pullback = curr_ao - pullback_cushion

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_ask_open)
                  stop_loss = round(pullback, 5)
                  # stop_loss = round(pullback - pullback_cushion, 3)

                  if stop_loss < open_price:
                      curr_pips_to_risk = open_price - stop_loss

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                            'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_buys += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh < ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_sells = True

            for trade in trades:
                if trade['trade_type'] != 'sell':
                    all_sells = False
                    break

            if all_sells:
              pullback = curr_bo + pullback_cushion

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_bid_open)
                  stop_loss = round(pullback, 5)
                  # stop_loss = round(pullback + pullback_cushion, 3)

                  if stop_loss > open_price:
                      curr_pips_to_risk = stop_loss - open_price

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                  'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                  'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_sells += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        if process_trade:
          iterate_thru_small_data(trades[-1])

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [18]:
reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_regular_simulation(2, 20, 5, 30, 3)

print(reward)
print('Num buys: ' + str(n_sells))
print('Num sells: ' + str(n_buys))
print('Num trades: ' + str(n_buys + n_sells))
print('Num wins: ' + str(n_wins))
print('Num losses: ' + str(n_losses))
print('Win streak: ' + str(win_streak))
print('Loss streak: ' + str(loss_streak))
print('Average pips risked: ' + str(sum(pips_risked) / len(pips_risked)))

-716.0000000000007
Num buys: 3
Num sells: 7
Num trades: 10
Num wins: 1
Num losses: 9
Win streak: 1
Loss streak: 9
Average pips risked: 0.0020000000000000018


In [19]:
# class AatTuple:
#     def __init__(self, time, pips_risked, pips_gained, total_profit, adx_assumption, ema_assumption, rsi_assumption):
#         self.time = time
#         self.pips_risked = pips_risked
#         self.pips_gained = pips_gained
#         self.total_profit = total_profit
#         self.adx_assumption = adx_assumption
#         self.ema_assumption = ema_assumption
#         self.rsi_assumption = rsi_assumption

x_train = []
for tup in training_data:
    curr_x = [tup.time, tup.pips_risked, tup.pips_gained, tup.total_profit, tup.adx_assumption, tup.ema_assumption, tup.rsi_assumption]
    x_train.append(curr_x)

x_train = np.array(x_train)


def distance_func(x, y):
    time_dist = 1000 * abs(x[0] - y[0])
    curr_pips_dist = 2 * abs(x[2] - y[2])
    pips_risked_dist = 3 * abs(x[1] - y[1])
    adx_dist = abs(x[4] - y[4])
    ema_dist = abs(x[5] - y[5])
    rsi_dist = abs(x[6] - y[6])

    return sum([time_dist, curr_pips_dist, pips_risked_dist, adx_dist, ema_dist, rsi_dist])

knn = NearestNeighbors(n_neighbors=15, metric=distance_func)
knn.fit(x_train)

NearestNeighbors(metric=<function distance_func at 0x129f69d40>, n_neighbors=15)

In [20]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_test_simulation(risk_reward_ratio, pullback_cushion, bar_length, adx_cutoff, spread_cutoff, max_open_trades=1, lam=0.5):
    pullback_cushion /= 10000
    spread_cutoff /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trades = deque(maxlen=max_open_trades)
    estimated_gains = []

    def iterate_thru_small_data(trade):
        def get_end_date(j):
          return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        j = df_small[df_small.Date >= trade['start_date']].index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak

        current_tuples = []
        step = 0

        while j < len(df_small):
          curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_date, curr_adx, curr_ema, curr_rsi, curr_ml, curr_mh = \
          df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                         'Ask_Close', 'Date', 'adx', 'ema200', 'rsi', 'Mid_Low', 'Mid_High']]

          if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount >= 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          # if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
          #     trade['stop_loss'] = curr_bid_high - trade['pips_risked']

          if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount >= 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount >= 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount >= 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          # if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
          #   trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

          if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount >= 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount >= 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          pips_gained_so_far = (curr_bid_close - trade['open_price']) * trade['n_units'] if trade['trade_type'] == 'buy' else (trade['open_price'] - curr_ask_close) * trade['n_units']

          z_adx_prev = 0 if len(current_tuples) == 0 else current_tuples[-1].adx_assumption * lam
          z_ema_prev = 0 if len(current_tuples) == 0 else current_tuples[-1].ema_assumption * lam
          z_rsi_prev = 0 if len(current_tuples) == 0 else current_tuples[-1].rsi_assumption * lam

          adx_holds = int(curr_adx > adx_cutoff)
          ema_holds = int(curr_ml > curr_ema if trade['trade_type'] == 'buy' else curr_mh < curr_ema)
          rsi_holds = int(curr_rsi < 70 if trade['trade_type'] == 'buy' else curr_rsi > 30)

          adx_assumption = z_adx_prev + ((1 - lam) * adx_holds)
          ema_assumption = z_ema_prev + ((1 - lam) * ema_holds)
          rsi_assumption = z_rsi_prev + ((1 - lam) * rsi_holds)

          current_tuple = np.array([step, trade['pips_risked'], pips_gained_so_far, 0, adx_assumption, ema_assumption, rsi_assumption])
          distances, neighbor_indices = knn.kneighbors(current_tuple.reshape(1, -1), 15)
          estimated_gain = 0
          distance_sum = 0

          for p in range(len(neighbor_indices[0])):
            l = neighbor_indices[0][p]
            l_dist = distances[0][p]
            estimated_gain += (x_train[l][3] / l_dist)
            distance_sum += 1 / l_dist

          estimated_gain /= distance_sum
          estimated_gains.append(estimated_gain)

          day_fees_now = calculate_day_fees(trade['start_date'], curr_date)

          if estimated_gain < pips_gained_so_far + day_fees_now:
            trade_amount = pips_gained_so_far * value_per_pip
            reward += trade_amount
            day_fees += day_fees_now

            n_wins += 1 if trade_amount >= 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount >= 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          step += 1
          j += 1

    i = 2

    ten_percent = int(len(df) / 10)

    while i < len(df):  
        if i % ten_percent == 0:
          print(i)        
        curr_date = df.loc[df.index[i], 'Date']

        # Remove any expired trades -----------------------------------------
        trades_to_remove = []

        for trade in trades:
          if trade['end_date'] is not None and curr_date >= trade['end_date']:
            trades_to_remove.append(trade)

        for trade in trades_to_remove:
          trades.remove(trade)
        # -------------------------------------------------------------------

        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        # enough_volatility = (spread / atr) <= atr_percentage
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        trending = adx >= adx_cutoff

        process_trade = False

        if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml > ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_buys = True

            for trade in trades:
                if trade['trade_type'] != 'buy':
                    all_buys = False
                    break

            if all_buys:
              pullback = curr_ao - pullback_cushion

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_ask_open)
                  stop_loss = round(pullback, 5)
                  # stop_loss = round(pullback - pullback_cushion, 3)

                  if stop_loss < open_price:
                      curr_pips_to_risk = open_price - stop_loss

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                            'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_buys += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh < ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_sells = True

            for trade in trades:
                if trade['trade_type'] != 'sell':
                    all_sells = False
                    break

            if all_sells:
              pullback = curr_bo + pullback_cushion

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_bid_open)
                  stop_loss = round(pullback, 5)
                  # stop_loss = round(pullback + pullback_cushion, 3)

                  if stop_loss > open_price:
                      curr_pips_to_risk = stop_loss - open_price

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                  'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                  'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_sells += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        if process_trade:
          iterate_thru_small_data(trades[-1])

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, estimated_gains

In [21]:
reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, estimated_gains = run_test_simulation(2, 20, 5, 30, 3)

print(reward)
print('Num buys: ' + str(n_sells))
print('Num sells: ' + str(n_buys))
print('Num trades: ' + str(n_buys + n_sells))
print('Num wins: ' + str(n_wins))
print('Num losses: ' + str(n_losses))
print('Win streak: ' + str(win_streak))
print('Loss streak: ' + str(loss_streak))
print('Average pips risked: ' + str(sum(pips_risked) / len(pips_risked)))

2463
4926
7389
9852
12315
14778
17241
19704


/Users/mymac/Forex_Robot/simulations/venv/lib/python3.7/site-packages/ipykernel_launcher.py:161: RuntimeWarning: divide by zero encountered in double_scalars
/Users/mymac/Forex_Robot/simulations/venv/lib/python3.7/site-packages/ipykernel_launcher.py:162: RuntimeWarning: divide by zero encountered in double_scalars
/Users/mymac/Forex_Robot/simulations/venv/lib/python3.7/site-packages/ipykernel_launcher.py:164: RuntimeWarning: invalid value encountered in double_scalars


22167
24630
-70.99999999999883
Num buys: 3
Num sells: 8
Num trades: 11
Num wins: 1
Num losses: 10
Win streak: 1
Loss streak: 9
Average pips risked: 0.0020000000000000018


In [22]:
sum(estimated_gains) / len(estimated_gains)

nan

In [23]:
min(estimated_gains)

-776.3783259694904

In [24]:
max(estimated_gains)

-635.0098373072924